In [ ]:
import imageio.v3 as iio
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
from cellSAM import cellsam_pipeline
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import sys
sys.path.append(r"C:\Users\Leina School\Desktop\Work Exp BDI\BDI-Nuclei_Segmentation\pipeline")

image = np.array(Image.open(r"C:\Users\Leina School\Desktop\Work Exp BDI\data\GTEX-113JC-2226.jpg")) 


In [ ]:
from img_manipulation import format_img, tile_img, seperate_tile_types
from find_means import find_means
from run_cellSAM import cellSAM


In [ ]:
w_target, h_target = 55750, 40500
tile_size = 250

cropped_image = format_img(image, w_target, h_target)

tiles = tile_img(cropped_image, tile_size)

means = find_means(tiles)

tissue_tiles, background_tiles = seperate_tile_types(means)

mask = cellSAM(tiles[tissue_tiles[967]])

In [ ]:
plt.imshow(mask)

In [ ]:
plt.imshow(tiles[tissue_tiles[967]])

In [ ]:
#self tiler
#crop image so exact tiles made
targ_width, targ_height = 55750, 40500

image = image[:targ_height, :targ_width]
#cropped_pil_image = Image.fromarray(cropped_image)
#cropped_pil_image.save('cropped_image.jpg')

In [ ]:

tile_size = 250

h_tile_num = image.shape[0]//tile_size
w_tile_num = image.shape[1]//tile_size

tiles = []

for y in range(h_tile_num):
    for x in range(w_tile_num):
        #start:stop(not inclusive)
        tile = image[y*tile_size:(y+1)*tile_size, x*tile_size:(x+1)*tile_size, :]
        tiles.append(tile)
            
#print(tiles)

In [ ]:
#histogram
means = []

#find averages of each tile pixel values in greyscale
for tile in tiles:
    img_section = Image.fromarray(tile)
    grey_img = img_section.convert("L")
    grey_tile = np.array(grey_img)
    means.append(grey_tile.mean()) 

print(means)
plt.hist(means, bins = 250)

In [ ]:
#seperate tile types
background_tile = []
tissue_tile = []

#for every tile, check avg, if surpasses threshhold then store index in array
for tile in range(len(means)):
   print(means[tile])
   if means[tile] < 225:
      tissue_tile.append(tile)
   else:
      background_tile.append(tile)

In [ ]:
image = tiles[tissue_tile[967]]

 #Run inference pipeline
mask = cellsam_pipeline(
    image, use_wsi=True, low_contrast_enhancement=False, gauge_cell_size=False
)

# Visualize results
plt.imshow(mask)

In [ ]:
plt.imshow(image)

In [ ]:
#iterate through tissue_tiles and create a tiles equivalent array of masks

masks = []

for tile in tqdm(tissue_tile):
    mask = cellsam_pipeline(tiles[tile], use_wsi=True, low_contrast_enhancement=False, gauge_cell_size=False)
    mask = mask.tolist()
    masks.append(mask)

In [ ]:
# Use several iterations to make the mask edge visible when plotting
dilated_mask = sp.ndimage.binary_dilation(pred_mask > 0, iterations=5)
edges = (dilated_mask > pred_mask).astype(np.uint8)

# Plot the image
plt.imshow(red_test_tile, cmap="gray")

# And the outlines from the mask
plt.imshow(255 * edges, cmap="Reds", alpha=edges)

In [ ]:
print(edges)